In [1]:
PROC = 'Procedure'

In [2]:
PROC_1 = "Proc. 1"# (Conventional detector training)"
PROC_2 = "Proc. 2"# (Two-phase training)"
PROC_3 = "Proc. 3"# (Crop batch training)"

In [3]:
from pathlib import Path

import pandas as pd
from tqdm.auto import tqdm
import wandb


res_path = Path('training_details.pkl')
LAZY = True


KEYS = [
    'lr',
    'num_epochs',
    'backbone_name',
    'crop_box_size',
    'k_fold_val_fold',
    'trainable_layers',
    'id_embedding_size',
    'use_crop_batch_inputs',
    'use_split_detect_recog',
]


def get_results():
    api = wandb.Api()

    SWEEPS = {
        "sandbox_experiments/ae74qmol": PROC_1,
        "sandbox_experiments/6wecie9y": PROC_2,
        "experiments/dyuqidka": PROC_3,
        "experiments/yenmkqwi": PROC_3,
    }

    configs = []

    for sweep_suffix, proc_name in tqdm(SWEEPS.items()):
        sweep = api.sweep(f"jpdr/{sweep_suffix}")
        runs = sweep.runs

        for run in tqdm(runs, leave=False):
            config = {k: run.config[k] for k in KEYS}
            config['name'] = proc_name
            configs.append(config)

    return pd.DataFrame(configs)


if not res_path.exists() or not LAZY:
    df = get_results()
    df.to_pickle(res_path)
else:
    df = pd.read_pickle(res_path)

df.loc[~df['use_crop_batch_inputs'], 'crop_box_size'] = None
df = df[df['crop_box_size'] != 'mean'].reset_index(drop=True)

In [4]:
df.to_csv('training_details.csv')